# Simulation of agents

## random agent
**Eg.1** Create a **random agent** and draw its trajectory

In [5]:
from scenario import *
from time import sleep
from random import random
scen = Scenario()
cube=Cube()
cube.position=Vector3(1,1,0.5)
cube.color=Color(r=1)
cube.rotation=Rotation.eular(z=0.3)
traj=Line()
traj.color=Color(b=1)
traj.is_arrow=True
traj.width=2
scen.add(cube,traj)
while scen.t<6:
    cube.local_velocity=Vector3.rand(x=[0,4])
    cube.local_angular_velocity=Rotation.eular(z=random()-0.4)
    traj.points.append(cube.position.tolist())
    scen.step(0.1)
    scen.render()
    sleep(0.1)

**Eg.2** Create ten random agents

In [13]:
from scenario import *
from time import sleep,time
from random import random
import types
scen = Scenario()
n=10
def on_step(obj):
    obj.local_velocity=Vector3.rand(x=[0,4])
    obj.local_angular_velocity=Rotation.eular(z=random()-0.5)
for i in range(n):
    cube=Cube()
    cube.position=Vector3(0,0,0.5)
    cube.color=Color.rand()
    cube.on_step=types.MethodType(on_step,cube)
    scen.add(cube)
t=time()
while scen.t<10:
    scen.step(0.1)
    scen.render()
    sleep(0.1)
print(time()-t)

22.59214758872986


**Eg.3** An agent follow a random agent

In [1]:
from scenario import *
from time import sleep
from random import random
scen = Scenario()
cube=Cube()
cube.scale=Vector3(2,1,1)
cube.position=Vector3(3,3,0.5)
cube.color=Color(r=1)
cube.rotation=Rotation.eular(z=0.3)
cube1=Cube()
cube1.scale=Vector3(2,1,1)
cube1.position=Vector3(0,0,0.5)
cube1.color=Color(b=1)
scen.add(cube,cube1)
scen.render()

Start to run

In [ ]:
while scen.t<10:
    cube.local_velocity=Vector3.rand(x=[0,2])
    cube.local_angular_velocity=Rotation.eular(z=random()-0.3)
    cube1.local_velocity=Vector3(x=1)
    cube1.direction_to(cube.position)
    scen.step(0.1)
    scen.render()
    sleep(0.1)

**Eg.4** An random agent and following agents, one by one.

In [15]:
from scenario import *
from time import sleep
from random import random

class Follower(Cube):
    def __init__(self):
        Cube.__init__(self)
        self.scale=Vector3(2,1,1)
        self.color=Color.rand()
        self.front=None
    def on_step(self):
        if self.front:
            d=(self.front.position-self.position).norm()
            self.local_velocity=Vector3(x=d*0.1)
            self.lookat(self.front.position)
        else:
            self.local_velocity=Vector3.rand(x=[0,2])
            self.local_angular_velocity=Rotation.eular(z=random()-0.3)
scen = Scenario()
n=10
front=None
for i in range(n):
    f=Follower()
    f.position=Vector3(10,0,0.5)-i*Vector3(3,0,0)
    if front:
        f.front=front
    scen.add(f)
    front=f
scen.render()

Start to run

In [16]:
while scen.t<20:
    scen.step(0.1)
    scen.render()
    sleep(0.1)